In [1]:
import json
import requests
import numpy as np
import pandas as pd
import time
import datetime

In [2]:
pd.set_option('display.max_rows', 20)

# Choose features

In [3]:
with open('match_details_total.json', 'r') as js:
     games = json.load(js)

### gameId

In [4]:
len(games.keys())

5985

In [5]:
gameId = '2994319360'
games[gameId]['gameId']

2994319360

### teams

#### win

In [6]:
wins = dict()
for j in range(2):
    wins[games[gameId]['teams'][j]['teamId']] = games[gameId]['teams'][j]['win']
wins

{100: 'Fail', 200: 'Win'}

#### ban

In [7]:
bans = dict()
for j in range(2):
    ban_champion_list =[]
    for i in range(5):
        # 벤을 하나도 안한 경기도 있을 수 있다.
        try:
            ban_champion_list.append(
                games[gameId]['teams'][j]['bans'][i]['championId'])
        except:
            continue
    bans[games[gameId]['teams'][j]['teamId']] = ban_champion_list
bans

{100: [8, 429, 421, 497, 60], 200: [114, 7, 38, 429, 31]}

### participants

#### pick & pick with role

In [8]:
picks = dict()
pick_details = dict()
pick_detail_teamId100 = dict()
pick_detail_teamId200 = dict()
teamId100 = []
teamId200 = []
for i in range(10):
    if games[gameId]['participants'][i]['teamId'] == 100:
        teamId100.append(games[gameId]['participants'][i]['championId'])
        pick_detail_teamId100[games[gameId]['participants'][i]['timeline']['lane'] +'_' + 
                              games[gameId]['participants'][i]['timeline']['role']] = \
                                  games[gameId]['participants'][i]['championId']
    else:
        teamId200.append(games[gameId]['participants'][i]['championId'])
        pick_detail_teamId200[games[gameId]['participants'][i]['timeline']['lane'] +'_' + 
                              games[gameId]['participants'][i]['timeline']['role']] = \
                                  games[gameId]['participants'][i]['championId']
picks[100] = teamId100
picks[200] = teamId200
pick_details[100] = pick_detail_teamId100
pick_details[200] = pick_detail_teamId200
print(picks)
print(pick_details)

{100: [18, 245, 48, 69, 150], 200: [202, 103, 113, 12, 157]}
{100: {'BOTTOM_DUO_CARRY': 18, 'JUNGLE_NONE': 245, 'BOTTOM_DUO_SUPPORT': 48, 'MIDDLE_SOLO': 69, 'TOP_SOLO': 150}, 200: {'BOTTOM_DUO_CARRY': 202, 'MIDDLE_SOLO': 103, 'JUNGLE_NONE': 113, 'BOTTOM_DUO_SUPPORT': 12, 'TOP_SOLO': 157}}


# Gather data

mapId=11 이 소환사의 협곡임

In [9]:
gather_data = dict()
for gameId in list(games.keys()):
    if games[gameId]['mapId']==11:
        gather_features = dict()
        # win
        wins = dict()
        for j in range(2):
            wins[games[gameId]['teams'][j]['teamId']] = games[gameId]['teams'][j]['win']
        # ban
        bans = dict()
        for j in range(2):
            ban_champion_list =[]
            for i in range(5):
                # 벤을 하나도 안한 경기도 있을 수 있다.
                try:
                    ban_champion_list.append(
                        games[gameId]['teams'][j]['bans'][i]['championId'])
                except:
                    continue
            bans[games[gameId]['teams'][j]['teamId']] = ban_champion_list
        # pick & pick&role
        picks = dict()
        pick_details = dict()
        pick_detail_teamId100 = dict()
        pick_detail_teamId200 = dict()
        teamId100 = []
        teamId200 = []
        for i in range(10):
            if games[gameId]['participants'][i]['teamId'] == 100:
                teamId100.append(games[gameId]['participants'][i]['championId'])
                pick_detail_teamId100[games[gameId]['participants'][i]['timeline']['lane'] +'_' + 
                                      games[gameId]['participants'][i]['timeline']['role']] = \
                                          games[gameId]['participants'][i]['championId']
            else:
                teamId200.append(games[gameId]['participants'][i]['championId'])
                pick_detail_teamId200[games[gameId]['participants'][i]['timeline']['lane'] +'_' + 
                                      games[gameId]['participants'][i]['timeline']['role']] = \
                                          games[gameId]['participants'][i]['championId']
        picks[100] = teamId100
        picks[200] = teamId200
        pick_details[100] = pick_detail_teamId100
        pick_details[200] = pick_detail_teamId200
        # gather data
        gather_features['win'] = wins
        gather_features['ban'] = bans
        gather_features['pick'] = picks
        gather_features['pick_role'] = pick_details
        gather_data[gameId] = gather_features
len(gather_data)

5264

In [10]:
gather_data[gameId]

{'ban': {100: [38, 81, 60, 497, 268], 200: [126, 12, 421, -1, 114]},
 'pick': {100: [96, 113, 90, 63, 24], 200: [59, 110, 80, 134, 412]},
 'pick_role': {100: {'BOTTOM_DUO_CARRY': 96,
   'BOTTOM_DUO_SUPPORT': 63,
   'JUNGLE_NONE': 113,
   'MIDDLE_SOLO': 90,
   'TOP_SOLO': 24},
  200: {'BOTTOM_DUO_CARRY': 110,
   'BOTTOM_DUO_SUPPORT': 412,
   'JUNGLE_NONE': 59,
   'MIDDLE_SOLO': 134,
   'TOP_SOLO': 80}},
 'win': {100: 'Win', 200: 'Fail'}}

In [12]:
with open('analysis_dataset.json', 'w') as js:
     json.dump(gather_data, js)